In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import numpy as np
import pandas as pd
from func import path, path_img, get_data, show_images_horizontally, show_image
from func_torch import ShopeeDataset, device, torch, f1_score_cal
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import gc
import faiss

In [2]:
# input
df = get_data(path / 'train.csv', path_img)
df.head()

,posting_id,image,image_phash,title,label_group,filepath,target,title_edit
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_129225211, train_2278313361]",paper bag victoria secret
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3423213080, train_3386243561]","double tape 3m vhb 12 mm x 4,5 m original / do..."
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_2288590299, train_3803689425]",maling tts canned pork luncheon meat 397 gr
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3342059966, train_2406599165]",daster batik lengan pendek - motif acak / camp...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3369186413, train_921438619]",nescafe \xc3\x89clair latte 220ml


In [3]:
dataset_data = ShopeeDataset(csv=df, train=True)
data_loader = DataLoader(dataset_data, batch_size=16, num_workers=4)

print(f"Dataset Len: {len(dataset_data):,}\nImage Shape [0]: {dataset_data[0][0].shape}")

Dataset Len: 34,250
Image Shape [0]: torch.Size([3, 512, 512])


In [4]:
model_name = 'efficientnet-b3'
image_embeddings_path = path / f'image_embed_{model_name}_pretrain.npy'
rerun = False

if rerun:
    # Run
    model_effnet = EfficientNet.from_name(model_name).to(device)

    print(model_name, device)

    embeddings = []
    with torch.no_grad():
        for image, label in tqdm(data_loader):
            image = image.to(device)
            img_embeddings = model_effnet(image)
            img_embeddings = img_embeddings.detach().cpu().numpy()
            embeddings.append(img_embeddings)
    all_image_embeddings = np.concatenate(embeddings)

    # Save
    np.save(image_embeddings_path, all_image_embeddings)

    # Clean memory
    del model_effnet
    _ = gc.collect()

else:
    # load
    all_image_embeddings = np.load(image_embeddings_path)

print(f"image_embeddings shape: {all_image_embeddings.shape}")

image_embeddings shape: (34250, 1000)


In [5]:
k_num = 10
nb, d = all_image_embeddings.shape 

if res := faiss.StandardGpuResources():
    print('gpu')
    index_flat = faiss.IndexFlatL2(d)
    index = faiss.index_cpu_to_gpu(res, 0, index_flat)
    index.add(all_image_embeddings)
else: 
    print('cpu')
    index = faiss.IndexFlatL2(d)
    index.add(all_image_embeddings)
print(index.is_trained, index.ntotal)

gpu
True 34250


In [10]:
target = df['target'].values.tolist()
scores, plot_check, f1_score_lst, pred_lst = {}, {}, [], {}

predictions = []
distances, indices = index.search(all_image_embeddings, k_num)
# ids = indices.flatten()
# predictions.append(df['posting_id'].iloc[ids].values.tolist())
# f1_score_lst = [f1_score_cal(i, v) for i, v in zip(target, predictions)]

# score = np.mean(f1_score_lst)
# print(f'--- F1 score for threshold is {score:.3f} ---\n')

# df['img_pred'] = predictions
# df['f1_score'] = score

100%|██████████████████████████████████████████████████████████████████████████| 34250/34250 [00:20<00:00, 1638.96it/s]


NameError: name 'num' is not defined

In [18]:
distances, indices = index.search(all_image_embeddings, k_num)

In [28]:
indices.shape

(34250, 10)

In [34]:
result = {}
for i in range(indices.shape[0]):
    base, neighbor = indices[i][0], indices[i][1:]
    df.iloc[base]['posting_id'].to_numpy()[1:].tolist()
    break

AttributeError: 'str' object has no attribute 'to_numpy'

In [38]:
df.iloc[neighbor]['posting_id']

30496    train_3846947656
14932    train_2325786285
2697     train_2204289747
28121    train_2793039505
25728    train_4269114602
1367      train_181749440
34173     train_188164388
20642     train_746062062
15122    train_1281072972
Name: posting_id, dtype: object

In [27]:
df.iloc[indices[0]]

,posting_id,image,image_phash,title,label_group,filepath,target,title_edit
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_129225211, train_2278313361]",paper bag victoria secret
30496,train_3846947656,e3fd69b5a82c522388b1b8c9c4a9b368.jpg,fa8266a38137cc3d,\xe2\xad\x90 Azzahra Gamis Syari Maxi Dress gbmdw,3641991970,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_698301312, train_3846947656, train_7499...",\xe2\xad\x90 azzahra gamis syari maxi dress gbmdw
14932,train_2325786285,709d1a3456d215b0b962bdcfda4def44.jpg,cd9ced7887e0c303,[DZ] PA024 - 1 PAKET CABLE PROTECTOR PELINDUNG...,2106621791,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_126355496, train_2325786285]",[dz] pa024 - 1 paket cable protector pelindung...
2697,train_2204289747,14968c490c496f06072ce3448dc27be3.jpg,9697c7696b603a31,[6pcs] SAPUTANGAN PRIA | SAPU TANGAN GAJAH | L...,175908741,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_2204289747, train_2260625403]",[6pcs] saputangan pria | sapu tangan gajah | l...
28121,train_2793039505,d28043bf22e7bcc5258bd3a3f9c3da34.jpg,f831ce2ab1c3b5c2,Mega Bank SBMPTN Soshum / saintek,4244588097,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_162101721, train_2279378364, train_4028...",mega bank sbmptn soshum / saintek
25728,train_4269114602,c05d3981445174e65d8ebaf150344ac7.jpg,beccc1b336ec90c1,ZJIANG 5809 PRO MINI BLUETOOTH THERMAL PRINTER...,3790388427,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_579594106, train_773741316, train_42691...",zjiang 5809 pro mini bluetooth thermal printer...
1367,train_181749440,0ad66c572a09c77699d3402f6544b35d.jpg,9314ee62cf299c66,Lactogrow 3/4 750gr Madu/Vanila/Plain,1285119273,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_146733663, train_3630522412, train_2701...",lactogrow 3/4 750gr madu/vanila/plain
34173,train_188164388,ff6a9d9eca41ccf523bb8e36f44a7d97.jpg,9aede511e11aad0e,[PROMO TERBATAS] Etude House Drawing Eyebrow N...,236048956,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_1097776268, train_188164388, train_2166...",[promo terbatas] etude house drawing eyebrow n...
20642,train_746062062,9aeb9685269259503be7faa9c42b6e85.jpg,d0e46b1ac67de1d0,LIVE SHOPEE Rok tutu One set Rajut,226718354,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_1163846796, train_746062062]",live shopee rok tutu one set rajut
15122,train_1281072972,71f3860939f216081f8c1533ce7d9b58.jpg,ac61d3999b3c8c36,Tango Wafer Vanilla 7gr x 20,3220256675,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3436262848, train_2152304807, train_301...",tango wafer vanilla 7gr x 20


In [11]:
df.head()

,posting_id,image,image_phash,title,label_group,filepath,target,title_edit
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_129225211, train_2278313361]",paper bag victoria secret
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3423213080, train_3386243561]","double tape 3m vhb 12 mm x 4,5 m original / do..."
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_2288590299, train_3803689425]",maling tts canned pork luncheon meat 397 gr
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3342059966, train_2406599165]",daster batik lengan pendek - motif acak / camp...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3369186413, train_921438619]",nescafe \xc3\x89clair latte 220ml


In [ ]:
test_id = 'train_2288590299'
subset = df.query(f'posting_id == "{test_id}"')[['filepath', 'title']]
tuples = tuple(subset.to_numpy()[0])
show_image(tuples)

tmp = pd.DataFrame(df.query(f'posting_id == "{test_id}"')['img_pred'].item(), columns=['posting_id'])
tmp = tmp.merge(df[['posting_id', 'filepath', 'title']], how='left', on='posting_id')
tuples = [tuple(i) for i in tmp[['filepath', 'title']].to_numpy()]

show_images_horizontally(tuples)